# 02. Schema Evolution

Iceberg의 **Schema Evolution** 기능을 통해 테이블 스키마를 안전하게 변경하고, 기존 데이터에 미치는 영향을 확인합니다.

## Schema Evolution 개념

전통적인 테이블 포맷에서 스키마 변경은 위험한 작업입니다.  
컬럼 이름이나 순서에 의존하기 때문에, 변경 시 기존 데이터와 호환성 문제가 발생할 수 있습니다.

### Iceberg의 접근 방식: Field ID 기반 매핑

Iceberg는 **이름(name)**이 아닌 **고유 ID(field ID)**로 컬럼을 추적합니다.

```
Schema v0:                    Schema v1:
┌─────────────────────┐      ┌─────────────────────────────┐
│ id=1  order_id      │      │ id=1  order_id              │
│ id=2  customer_id   │      │ id=2  customer_id           │
│ id=3  product_name  │      │ id=3  item_name (renamed!)  │
│ id=4  order_date    │      │ id=4  order_date            │
│ id=5  amount        │      │ id=5  amount                │
│ id=6  status        │      │ id=6  status                │
└─────────────────────┘      │ id=7  discount_rate (new!)  │
                              └─────────────────────────────┘
```

- 컬럼 이름을 변경해도 `field ID`가 같으면 동일 컬럼으로 인식
- 새 컬럼이 추가되면 기존 데이터에서는 `NULL`로 반환
- 기존 Parquet 파일은 **절대 다시 쓰지 않음** (read-time projection)

### 지원되는 스키마 변경

| 변경 유형 | SQL | 설명 |
|-----------|-----|------|
| 컬럼 추가 | `ADD COLUMNS` | 새 컬럼 추가 (기존 데이터는 NULL) |
| 컬럼 이름 변경 | `RENAME COLUMN` | field ID는 유지 |
| 컬럼 타입 변경 | `ALTER COLUMN ... TYPE` | 호환 가능한 타입만 (int→bigint 등) |
| 컬럼 삭제 | `DROP COLUMN` | 메타데이터에서만 제거 |

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df

In [ ]:
spark = create_spark_session("SchemaEvolution")

## 테이블 생성 및 초기 데이터 삽입

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.schema_orders")

spark.sql("""
    CREATE TABLE demo.lab.schema_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

print("테이블 생성 완료")

In [ ]:
# 100건 삽입 (Schema v0)
orders = generate_orders(num_records=100, seed=42)
df = to_spark_df(spark, orders)
df.writeTo("demo.lab.schema_orders").append()

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.schema_orders").collect()[0]["cnt"]
print(f"초기 데이터 삽입: {count}건")

In [ ]:
# 초기 스냅샷 ID 저장
snap_v0 = spark.sql("SELECT snapshot_id FROM demo.lab.schema_orders.snapshots ORDER BY committed_at").collect()
snapshot_v0_id = snap_v0[-1]["snapshot_id"]
print(f"Schema v0 스냅샷 ID: {snapshot_v0_id}")

# 현재 스키마 확인
print("\n[ 현재 스키마 ]")
spark.sql("DESCRIBE demo.lab.schema_orders").show(truncate=False)

## ADD COLUMNS: 새 컬럼 추가

`discount_rate` 컬럼을 추가합니다. 기존 데이터에서 이 컬럼은 `NULL`이 됩니다.

In [ ]:
spark.sql("""
    ALTER TABLE demo.lab.schema_orders
    ADD COLUMNS (discount_rate DOUBLE)
""")

print("discount_rate 컬럼 추가 완료")

# 스키마 확인
print("\n[ 변경된 스키마 ]")
spark.sql("DESCRIBE demo.lab.schema_orders").show(truncate=False)

In [ ]:
# 기존 데이터에서 discount_rate 값 확인
print("[ 기존 데이터의 discount_rate 값 ]\n")

spark.sql("""
    SELECT order_id, product_name, amount, discount_rate
    FROM demo.lab.schema_orders
    ORDER BY order_id
    LIMIT 10
""").show(truncate=False)

null_count = spark.sql("""
    SELECT COUNT(*) AS cnt
    FROM demo.lab.schema_orders
    WHERE discount_rate IS NULL
""").collect()[0]["cnt"]

print(f"discount_rate가 NULL인 레코드: {null_count}건 (= 기존 전체 데이터)")

## 새 스키마로 데이터 삽입

`discount_rate`를 포함한 새 데이터 50건을 삽입합니다.

In [ ]:
import random
random.seed(77)

# discount_rate를 포함하는 데이터 생성
new_orders = generate_orders(num_records=50, id_offset=101, seed=77)
for order in new_orders:
    order['discount_rate'] = round(random.uniform(0.0, 0.3), 2)

# Spark DataFrame으로 변환 (discount_rate 포함)
import pandas as pd
from pyspark.sql.functions import col

pdf = pd.DataFrame(new_orders)
new_df = spark.createDataFrame(pdf)
new_df = new_df.withColumn("order_date", col("order_date").cast("date"))

new_df.writeTo("demo.lab.schema_orders").append()

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.schema_orders").collect()[0]["cnt"]
print(f"삽입 후 총 레코드: {count}건")

In [ ]:
# 새 데이터의 discount_rate 확인
print("[ 새로 삽입된 데이터 (discount_rate 포함) ]\n")

spark.sql("""
    SELECT order_id, product_name, amount, discount_rate
    FROM demo.lab.schema_orders
    WHERE order_id >= 101
    ORDER BY order_id
    LIMIT 10
""").show(truncate=False)

# NULL vs non-NULL 통계
stats = spark.sql("""
    SELECT
        COUNT(*) AS total,
        COUNT(discount_rate) AS has_discount,
        COUNT(*) - COUNT(discount_rate) AS null_discount
    FROM demo.lab.schema_orders
""").collect()[0]

print(f"전체: {stats['total']}건, discount_rate 있음: {stats['has_discount']}건, NULL: {stats['null_discount']}건")

## RENAME COLUMN: 컬럼 이름 변경

`product_name`을 `item_name`으로 변경합니다. Field ID는 그대로 유지됩니다.

In [ ]:
spark.sql("""
    ALTER TABLE demo.lab.schema_orders
    RENAME COLUMN product_name TO item_name
""")

print("product_name → item_name 변경 완료")

# 변경된 스키마 확인
print("\n[ 변경된 스키마 ]")
spark.sql("DESCRIBE demo.lab.schema_orders").show(truncate=False)

In [ ]:
# 이름 변경 후 데이터 조회 — 기존 데이터도 item_name으로 조회 가능
print("[ RENAME 후 데이터 조회 ]\n")

spark.sql("""
    SELECT order_id, item_name, amount
    FROM demo.lab.schema_orders
    ORDER BY order_id
    LIMIT 5
""").show(truncate=False)

print("기존 데이터도 새 이름(item_name)으로 정상 조회됩니다.")
print("→ Iceberg가 field ID로 매핑하기 때문입니다.")

## metadata.json에서 스키마 히스토리 확인

metadata.json의 `schemas` 배열에 모든 스키마 버전이 기록됩니다.

In [ ]:
import json
import glob
import os

METADATA_PATH = "/home/jovyan/data/warehouse/lab/schema_orders/metadata"

# 최신 metadata.json
metadata_files = sorted(glob.glob(f"{METADATA_PATH}/v*.metadata.json"))
latest = metadata_files[-1]
print(f"최신 metadata: {os.path.basename(latest)}")
print(f"전체 metadata 파일 수: {len(metadata_files)}개\n")

with open(latest) as f:
    meta = json.load(f)

print(f"현재 schema ID: {meta.get('current-schema-id')}")
print(f"스키마 버전 수: {len(meta.get('schemas', []))}\n")

for schema in meta.get('schemas', []):
    print(f"--- Schema ID: {schema['schema-id']} ---")
    for field in schema['fields']:
        print(f"  id={field['id']:2d}  {field['name']:20s}  type={field['type']}")
    print()

## Time Travel로 이전 스키마 데이터 조회

스키마가 변경되기 전 시점의 데이터를 Time Travel로 조회합니다.  
이전 스키마에 없는 `discount_rate`는 어떻게 처리될까요?

In [ ]:
# Schema v0 시점의 데이터 조회 (ADD COLUMNS 이전)
print(f"[ Schema v0 시점 (스냅샷 {snapshot_v0_id}) ]\n")

spark.sql(f"""
    SELECT *
    FROM demo.lab.schema_orders
    VERSION AS OF {snapshot_v0_id}
    ORDER BY order_id
    LIMIT 5
""").show(truncate=False)

v0_count = spark.sql(f"""
    SELECT COUNT(*) AS cnt
    FROM demo.lab.schema_orders
    VERSION AS OF {snapshot_v0_id}
""").collect()[0]["cnt"]

print(f"v0 시점 레코드 수: {v0_count}건")
print("\n→ 현재 스키마(item_name, discount_rate)가 이전 데이터에도 적용됩니다.")
print("→ discount_rate는 NULL, product_name은 item_name으로 표시됩니다.")

## 관찰 포인트

### Schema Evolution의 핵심

1. **컬럼이 추가되어도 기존 데이터는 그대로입니다**
   - 기존 Parquet 파일은 다시 쓰지 않습니다
   - 새 컬럼은 기존 데이터에서 `NULL`로 반환됩니다 (read-time projection)

2. **컬럼 이름을 변경해도 데이터가 깨지지 않습니다**
   - Iceberg는 `field ID`로 컬럼을 추적합니다
   - `product_name` → `item_name`으로 변경해도, ID가 같으면 동일 컬럼

3. **Iceberg가 스키마 버전을 추적합니다**
   - `metadata.json`의 `schemas` 배열에 모든 버전이 기록
   - `current-schema-id`로 현재 활성 스키마를 식별
   - Time Travel 시에도 현재 스키마 기준으로 데이터를 읽습니다

4. **기존 파일을 다시 쓰지 않는 장점**
   - 대규모 테이블에서 스키마 변경이 **즉시(O(1))** 완료
   - ALTER TABLE은 metadata.json만 갱신하므로 매우 빠름
   - 데이터 마이그레이션이 필요 없음

In [ ]:
spark.stop()
print("Spark 세션 종료")